In [26]:
!pip install face_recognition
!pip install dlib
!pip install imutils

In [27]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [28]:
import face_recognition
import cv2
import os 
import glob 
from google.colab.patches import cv2_imshow

In [29]:
class extract:
  
    def __init__(self, inUrl, outUrl=""):
        self.inUrl = "/content/drive/My Drive"+inUrl+"/*.jpg"
        self.outUrl = "/content/drive/My Drive"+outUrl
        print(self.inUrl, self.outUrl)
    
    def getImages(self):
        imag=list()
        for image in glob.glob(self.inUrl):
            img = cv2.imread(image)
            #  imResized = cv2.resize(img,(128,128))
            imag.append(img)
        return imag

    def storeImages(self, landmarks):
      try:
        for landmark in landmarks:
          path = self.outUrl+'/output/'+landmark
          fileName = landmark+str(landmarks[landmark][1])+'.jpg'
          try: 
            os.makedirs(path, exist_ok = True) 
          except OSError as error: 
            pass
          cv2.imwrite(os.path.join(path , fileName), landmarks[landmark][0])
        return True
      except:
        return False

    
    def extractNose(self, face_landmarks_list, image, p):
        slist1, flist2 = [], []
        for i in face_landmarks_list[0]["nose_bridge"]:
            slist1.append(i[1])
        for i in face_landmarks_list[0]["nose_tip"]:
            flist2.append(i[0])
        x1 = min(flist2)-p
        x2 = max(flist2)+p
        y1 = min(slist1)-p
        y2 = max(slist1)+p
        imgCrop = image[y1:y2,x1:x2]
        return imgCrop, 'nose'
    
    def extractLip(self, face_landmarks_list, image, p):
        flist1, slist1, flist2, slist2 = [], [], [], []
        for i in face_landmarks_list[0]["top_lip"]:
            flist1.append(i[0])
            slist1.append(i[1])
        for i in face_landmarks_list[0]["bottom_lip"]:
            flist2.append(i[0])
            slist2.append(i[1])
        x1=max(min(flist1), min(flist2))-p
        x2=max(max(flist1), max(flist2))+p
        y1=min(min(slist2), min(slist2))-p
        y2=max(max(slist1), max(slist2))+p
        imgCrop = image[y1:y2,x1:x2]
        return imgCrop, 'lip'
        
    def defaultExtract(self, face_landmarks_list, image, f, p):
        flist, slist = [], []
        for i in face_landmarks_list[0][f]:
            flist.append(i[0])
            slist.append(i[1])
        x1 = min(flist)-p
        x2 = max(flist)+p
        y1 = min(slist)-p
        y2 = max(slist)+p
        imgCrop = image[y1:y2,x1:x2]
        return imgCrop
        
    def extraction(self):
        j=1
        images = glob.glob(self.inUrl)
        outOf = len(images)
        for img in images:
            image = cv2.imread(img)
            face_landmarks_list = face_recognition.face_landmarks(image)
            try:
              fields={}
              for f in face_landmarks_list[0]:
                  imgCopy = image.copy()
                  if 'nose' in f:
                      imgCrop, f = self.extractNose(face_landmarks_list, imgCopy, 10)
                  elif 'lip' in f:
                      imgCrop, f = self.extractLip(face_landmarks_list, imgCopy, 10)
                  else:
                      imgCrop = self.defaultExtract(face_landmarks_list, imgCopy, f, 5)
                  fields[f] = [imgCrop, j]
            except:
              continue
            path=self.outUrl+'/input/'
            try: 
              os.makedirs(path, exist_ok = True) 
            except OSError as error: 
              pass
            flag = self.storeImages(fields)
            if flag:
              path=self.outUrl+'/input/'
              fileName= 'image'+str(j)+'.jpg'
              cv2.imwrite(os.path.join(path , fileName), image)
              print('\r', j, 'of', outOf, end="")
              j+=1
        print("\n Extraction Successful!!")

In [30]:
obj = extract("/dataset", "/datasetoutput")

/content/drive/My Drive/dataset/*.jpg /content/drive/My Drive/datasetoutput


In [ ]:
obj.extraction()

 118

In [ ]:
for i in ['chin', 'nose', 'chin', 'left_eye', 'right_eye', 'right_eyebrow', 'left_eyebrow']:
  obj1 = extract("/face1output/output/"+i)
  print(len(obj1.getImages()))

503
503
503
503
503
503
503


In [ ]:
obj2 = extract("/face1output/input")
print(len(obj2.getImages()))

525


In [13]:
import time
for i in range(10):
  print('\r', i, end='')
  time.sleep(1)

 9